# Lineage information


This example shows how lineage trees can be passed, specifically
useful for the {class}`~moscot.problems.time.LineageProblem`, which requires lineage information.

{mod}`moscot` allows this by passing the:
1. precomputed cost matrices,
2. barcode information
3. lineage tree as a {class}`~networkx.DiGraph`.

In this notebook, we consider the lineage tree case.

```{note}
- TODO: link to other examples
```

In [1]:
from moscot.datasets import simulate_data
from moscot.problems.time import LineageProblem

In [2]:
adata = simulate_data(n_distributions=3, key="day", quad_term="tree")
adata

AnnData object with n_obs × n_vars = 60 × 60
    obs: 'day', 'celltype'
    uns: 'trees'

We assume trees are saved in {attr}`anndata.AnnData.uns` as a {class}`dict`, where each key is a value in {attr}`anndata.AnnData.obs` and each value is a {class}`~networkx.DiGraph`.

In [3]:
adata.uns["trees"]

{0: <networkx.classes.digraph.DiGraph at 0x29c6d6140>,
 1: <networkx.classes.digraph.DiGraph at 0x29c6d5450>,
 2: <networkx.classes.digraph.DiGraph at 0x29c6d51e0>}

## Leaf distance

Now, we can instantiate and prepare the {class}`~moscot.problems.time.LineageProblem` by specifying the cost.

In [4]:
lp = LineageProblem(adata)
lp = lp.prepare(time_key="day", lineage_attr={"attr": "uns", "key": "trees", "cost": "leaf_distance"})

INFO     Computing pca with `n_comps=30` for `xy` using `adata.X`                                                  
WARNING  Cost matrix contains `NaN` values, setting them to the maximum value `3.0`.                               
WARNING  Cost matrix contains `NaN` values, setting them to the maximum value `3.0`.                               
INFO     Computing pca with `n_comps=30` for `xy` using `adata.X`                                                  
WARNING  Cost matrix contains `NaN` values, setting them to the maximum value `3.0`.                               
WARNING  Cost matrix contains `NaN` values, setting them to the maximum value `3.0`.                               


Internally, cost matrices have been computed from the trees using the shortest-path distance between the leaves. Let us investigate the first few entries of the cost matrix computed from the first lineage tree.

In [5]:
lp[0, 1].x.data_src[:3, :3]

array([[0., 2., 3.],
       [2., 0., 3.],
       [3., 3., 0.]])

Similarly, we investigate parts of the cost matrix created from the second tree.

In [6]:
lp[0, 1].y.data_src[:3, :3]

array([[0., 2., 3.],
       [2., 0., 3.],
       [3., 3., 0.]])

Note that the gene expression term is still saved as two point clouds. This cost matrix will be computed by the backend.

In [7]:
lp[0, 1].xy.data_src.shape, lp[0, 1].xy.data_tgt.shape

((20, 30), (20, 30))